In [2]:
import os
import numpy as np
import SimpleITK
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [6]:
brainT1_path = 'BRATS-2/Image_Data/HG/0001/VSD.Brain.XX.O.MR_T1/VSD.Brain.XX.O.MR_T1.685.mha'
brainT1c_path ='BRATS-2/Image_Data/HG/0001/VSD.Brain.XX.O.MR_T1c/VSD.Brain.XX.O.MR_T1c.686.mha'
brainT2_path = 'BRATS-2/Image_Data/HG/0001/VSD.Brain.XX.O.MR_T2/VSD.Brain.XX.O.MR_T2.687.mha'
brainFLAIR_path = 'BRATS-2/Image_Data/HG/0001/VSD.Brain.XX.O.MR_Flair/VSD.Brain.XX.O.MR_Flair.684.mha'
brainT1_img = SimpleITK.ReadImage(brainT1_path)
brainT1c_img = SimpleITK.ReadImage(brainT1c_path)
brainT2_img = SimpleITK.ReadImage(brainT2_path)
brainFlair_img = SimpleITK.ReadImage(brainFLAIR_path)
truth_path = 'BRATS-2/Image_Data/HG/0001/VSD.Brain_3more.XX.XX.OT/VSD.Brain_3more.XX.XX.OT.6560.mha'
truth_img = SimpleITK.ReadImage(truth_path)
initial_seg_path = 'BRATS-2/Image_Data/HG/0001/seg.nii.gz'
initial_seg = SimpleITK.ReadImage(initial_seg_path)
atlas_path = 'BRATS-2/Image_Data/HG/0001/Atlas/'

A method to display slices

In [3]:
def sitk_show(img, title=None, margin=0.0, dpi=40):
    nda = SimpleITK.GetArrayFromImage(img)
    #spacing = img.GetSpacing()
    figsize = (1 + margin) * nda.shape[0] / dpi, (1 + margin) * nda.shape[1] / dpi
    #extent = (0, nda.shape[1]*spacing[1], nda.shape[0]*spacing[0], 0)
    extent = (0, nda.shape[1], nda.shape[0], 0)
    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])
    plt.set_cmap("gray")
    ax.imshow(nda,extent=extent,interpolation=None)
    
    if title:
        plt.title(title)
    
    plt.show()

Adds as outlier voxels 3 standard deviations away from the norm

In [73]:
def add_outlier(seg, img, k):
    for i in range(1,k+1):
        stdev = img[seg==i].std()
        mean = img[seg==i].mean()
        sub = np.all([seg==i, np.any([img>mean+3*stdev, img<mean-3*stdev], 0)],0)
        print np.unique(sub)
        seg[sub] = 4
        
        

Takes an image segmentation (gray matter, white matter and CSF) and initializes theta. outliers are more than 3 standard deviations away from norm

In [5]:
def init_theta(seg,img, theta, k, c):
    for i in range(0,c):
        for j in range(1,k+2):
            theta[i][2*j] = img[i][seg==(j)].mean()
            theta[i][2*j+1] = img[i][seg==(j)].std()
            
            
    
        

In [6]:
def init_tumor(seg, tumor, c):
    for i in range(0,c):
        tumor[i][seg==4] = 1
            

In [24]:
def T_prob(Ti, alphai):
    return alphai**np.sum(Ti) * (1-alphai)**(Ti.size-np.sum(Ti))

In [20]:
def help_func(a,b, muk, stdevk, mukP1, stdevkP1):
    return mlab.normpdf(a,muk,stdevk)**(1-b) * mlab.normpdf(a,mukP1, stdevkP1)**b

def Y_prob(yi, ti, ki, theta):
    mu=[theta[0][2*ki],theta[1][2*ki],theta[2][2*ki],theta[3][2*ki]]
    stdev=[theta[0][2*ki+1],theta[1][2*ki+1],theta[2][2*ki+1],theta[3][2*ki+1]]
    mukP1 = [theta[0][-2],theta[1][-2],theta[2][-2],theta[3][-2]]
    stdevkP1 = [theta[0][-1],theta[1][-1],theta[2][-1],theta[3][-1]]
    return np.prod(map(help_func, yi, ti, mu, stdev, mukP1, stdevkP1))

In [9]:
def get_class(seg, probMap):
    return np.argmax([np.mean(probMap[seg==1]), np.mean(probMap[seg==2]), np.mean(probMap[seg==3])])+1

In [10]:
def Qi(yi,ti, k, theta, alphai, atlas, index):
    sum = 0
    for i in range(1,k+1): #3 classes are 1,2,3
        sum += Y_prob(yi, ti, i, theta)*T_prob(ti, alphai)*atlas[i][index]
    return sum

In [37]:
def Wik(yi, theta, atlas, ti, index, k):
    mu=[theta[0][2*k],theta[1][2*k],theta[2][2*k],theta[3][2*k]]
    stdev=[theta[0][2*k+1],theta[1][2*k+1],theta[2][2*k+1],theta[3][2*k+1]]
#     norm = map(mlab.normpdf, yi, mu, theta)
#     pow = norm**(1-ti)
#     prod = np.prod(pow)
#     return atlas[k][index]*prod
    return atlas[k][index]*np.prod(map(mlab.normpdf, yi, mu, stdev)**(1-ti))

In [12]:
def get_all_t(c): # returnes the 2^c configurations
    if c == 1:
        return [np.array([0]), np.array([1])]
    else:
        arr = get_all_t(c-1)
        return[np.append(a0,b0) for a0 in arr for b0 in np.array([0,1])]

Initialization of variables, and some loading that had to be done in order

In [13]:
k = 3 #3 tissue classes
c = 4 # my data has 4 channels
N = brainT1_img.GetHeight()*brainT1_img.GetWidth()*brainT1_img.GetDepth()
theta = np.empty([c, 2*(k+2)], double) # first 2 vals in a row are empty to match index's. class one is 2*1 and 2*1+1

img = np.empty([c,N])
img[0] = SimpleITK.GetArrayFromImage(brainT1_img).flatten()
img[1] = SimpleITK.GetArrayFromImage(brainT1c_img).flatten()
img[2] = SimpleITK.GetArrayFromImage(brainT2_img).flatten()
img[3] = SimpleITK.GetArrayFromImage(brainFlair_img).flatten()
initial_seg = SimpleITK.GetArrayFromImage(initial_seg).flatten()

atlas = np.empty((4,N), double) # atlas[0] is empty to make index's match with segmentation.
csf = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(atlas_path + "CSF_warped.nii.gz")).flatten()
gm = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(atlas_path + "GM_warped.nii.gz")).flatten()
wm = SimpleITK.GetArrayFromImage(SimpleITK.ReadImage(atlas_path + "WM_warped.nii.gz")).flatten()
atlas[get_class(initial_seg, csf)] = csf
# print "csf class: " + str(get_class(initial_seg, csf))
atlas[get_class(initial_seg, gm)] = gm
# print "gm class: " + str(get_class(initial_seg, gm))
atlas[get_class(initial_seg, wm)] = wm
# print "wm class: " + str(get_class(initial_seg, wm))

alpha = np.full(N, 0.3)
alpha[initial_seg==4] = 0.7

add_outlier(initial_seg, img[0], k)
tumor = np.zeros((c,N), int)
init_tumor(initial_seg, tumor, c)

init_theta(initial_seg, img, theta, k, c)
all_t = get_all_t(c)

#### EM, i.e. main segmentation

Considering only non-zero( in all channels) voxels

In [78]:
for i in np.nonzero(x):
    print i

[2 3 4 5 6]


E-Step

In [74]:
initial_seg_path = 'BRATS-2/Image_Data/HG/0001/seg.nii.gz'
initial_seg = SimpleITK.ReadImage(initial_seg_path)
initial_seg = SimpleITK.GetArrayFromImage(initial_seg).flatten()
N = brainT1_img.GetHeight()*brainT1_img.GetWidth()*brainT1_img.GetDepth()
img = np.empty([4,N])
img[0] = SimpleITK.GetArrayFromImage(brainT1_img).flatten()
img[1] = SimpleITK.GetArrayFromImage(brainT1c_img).flatten()
img[2] = SimpleITK.GetArrayFromImage(brainT2_img).flatten()
img[3] = SimpleITK.GetArrayFromImage(brainFlair_img).flatten()

In [77]:
np.unique(initial_seg)

array([0, 1, 2, 3, 4], dtype=uint32)

In [76]:
add_outlier(initial_seg, img[0], 3)

[False  True]
[False  True]
[False  True]


In [27]:
stdev = img[0][initial_seg==1].std()
mean = img[0][initial_seg==1].mean()
sub = [initial_seg==1] and [img[0] > mean+3*stdev] and [img[0] < mean - 3*stdev]

In [38]:
[False, True, False] and [True, True, True]

[True, True, True]

In [51]:
x=np.array([0,0,1,1,1,0,0])
y=np.array([1,2,3,6,2,3,6])
z=numpy.all([x==1, y>5], 0)

In [52]:
z[1]

False

In [53]:
x[z] = 2

In [72]:
np.any([[True,False],[True,True]], 0)

array([ True,  True], dtype=bool)